In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
sc.set_figure_params(dpi=100, frameon=False)
sc.logging.print_header()

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


scanpy==1.6.0 anndata==0.7.4 umap==0.4.6 numpy==1.19.2 scipy==1.6.1 pandas==1.2.3 scikit-learn==0.23.2 statsmodels==0.11.1 python-igraph==0.8.3 leidenalg==0.8.3


In [2]:
adata = sc.read('../datasets/sciplex3_old_reproduced.h5ad')

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
adata.obs['ct_dose'] = adata.obs.cell_type.astype('str') + '_' + adata.obs.dose_val.astype('str')

Round robin splits: dose and cell line combinations will be held out in turn.

In [4]:
i = 0
split_dict = {}

In [5]:
# single ct holdout
for ct in adata.obs.cell_type.unique():
    for dose in adata.obs.dose_val.unique():
        i += 1
        split_name = f'split{i}'
        split_dict[split_name] = f'{ct}_{dose}'
        
        adata.obs[split_name] = 'train'
        adata.obs[split_name][adata.obs.ct_dose == f'{ct}_{dose}'] = 'ood'
        
        test_idx = sc.pp.subsample(adata[adata.obs[split_name] != 'ood'], .16, copy=True).obs.index
        adata.obs[split_name].loc[test_idx] = 'test'
        
        display(adata.obs[split_name].value_counts())

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWa

train    229595
test      43732
ood       17561
Name: split1, dtype: int64

train    228515
test      43526
ood       18847
Name: split2, dtype: int64

train    229387
test      43692
ood       17809
Name: split3, dtype: int64

train    229731
test      43758
ood       17399
Name: split4, dtype: int64

...

train    214820
test      40917
ood       35151
Name: split8, dtype: int64

train    229287
test      43673
ood       17928
Name: split9, dtype: int64

train    227678
test      43367
ood       19843
Name: split10, dtype: int64

train    228686
test      43559
ood       18643
Name: split11, dtype: int64

train    230139
test      43835
ood       16914
Name: split12, dtype: int64

In [6]:
# double ct holdout
for cts in [('A549', 'MCF7'), ('A549', 'K562'), ('MCF7', 'K562')]:
    for dose in adata.obs.dose_val.unique():
        i += 1
        split_name = f'split{i}'
        split_dict[split_name] = f'{cts[0]}+{cts[1]}_{dose}'
        
        adata.obs[split_name] = 'train'
        adata.obs[split_name][adata.obs.ct_dose == f'{cts[0]}_{dose}'] = 'ood'
        adata.obs[split_name][adata.obs.ct_dose == f'{cts[1]}_{dose}'] = 'ood'
        
        test_idx = sc.pp.subsample(adata[adata.obs[split_name] != 'ood'], .16, copy=True).obs.index
        adata.obs[split_name].loc[test_idx] = 'test'
        
        display(adata.obs[split_name].value_counts())

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


train    199842
ood       52982
test      38064
Name: split13, dtype: int64

train    196536
ood       56917
test      37435
Name: split14, dtype: int64

train    198053
ood       55111
test      37724
Name: split15, dtype: int64

train    200204
ood       52550
test      38134
Name: split16, dtype: int64

...

train    215523
test      41052
ood       34313
Name: split20, dtype: int64

train    199533
ood       53349
test      38006
Name: split21, dtype: int64

train    195699
ood       57913
test      37276
Name: split22, dtype: int64

train    197353
ood       55945
test      37590
Name: split23, dtype: int64

train    200612
ood       52065
test      38211
Name: split24, dtype: int64

In [7]:
# triple ct holdout
for dose in adata.obs.dose_val.unique():
    i += 1
    split_name = f'split{i}'

    split_dict[split_name] = f'all_{dose}'
    adata.obs[split_name] = 'train'
    adata.obs[split_name][adata.obs.dose_val == dose] = 'ood'

    test_idx = sc.pp.subsample(adata[adata.obs[split_name] != 'ood'], .16, copy=True).obs.index
    adata.obs[split_name].loc[test_idx] = 'test'

    display(adata.obs[split_name].value_counts())

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying 

train    184782
ood       70910
test      35196
Name: split25, dtype: int64

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying 

train    179868
ood       76760
test      34260
Name: split26, dtype: int64

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying 

train    182393
ood       73754
test      34741
Name: split27, dtype: int64

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying 

train    185997
ood       69464
test      35427
Name: split28, dtype: int64

In [8]:
split_dict

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'split1': 'A549_0.1',
 'split2': 'A549_0.001',
 'split3': 'A549_0.01',
 'split4': 'A549_1.0',
 'split5': 'MCF7_0.1',
 'split6': 'MCF7_0.001',
 'split7': 'MCF7_0.01',
 'split8': 'MCF7_1.0',
 'split9': 'K562_0.1',
 'split10': 'K562_0.001',
 'split11': 'K562_0.01',
 'split12': 'K562_1.0',
 'split13': 'A549+MCF7_0.1',
 'split14': 'A549+MCF7_0.001',
 'split15': 'A549+MCF7_0.01',
 'split16': 'A549+MCF7_1.0',
 'split17': 'A549+K562_0.1',
 'split18': 'A549+K562_0.001',
 'split19': 'A549+K562_0.01',
 'split20': 'A549+K562_1.0',
 'split21': 'MCF7+K562_0.1',
 'split22': 'MCF7+K562_0.001',
 'split23': 'MCF7+K562_0.01',
 'split24': 'MCF7+K562_1.0',
 'split25': 'all_0.1',
 'split26': 'all_0.001',
 'split27': 'all_0.01',
 'split28': 'all_1.0'}

In [9]:
adata.uns['splits'] = split_dict

In [10]:
sc.write('../datasets/sciplex3_old_reproduced.h5ad', adata)

... storing 'ct_dose' as categorical
... storing 'split1' as categorical
... storing 'split2' as categorical
... storing 'split3' as categorical
... storing 'split4' as categorical
... storing 'split5' as categorical
... storing 'split6' as categorical
... storing 'split7' as categorical
... storing 'split8' as categorical
... storing 'split9' as categorical
... storing 'split10' as categorical
... storing 'split11' as categorical
... storing 'split12' as categorical
... storing 'split13' as categorical
... storing 'split14' as categorical
... storing 'split15' as categorical
... storing 'split16' as categorical
... storing 'split17' as categorical
... storing 'split18' as categorical
... storing 'split19' as categorical
... storing 'split20' as categorical
... storing 'split21' as categorical
... storing 'split22' as categorical
... storing 'split23' as categorical
... storing 'split24' as categorical
... storing 'split25' as categorical
... storing 'split26' as categorical
... storin